In [1]:
!pip install onnx onnxruntime num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=eaf8e09edb305e688ba7615153c553e3d430377dc97c9685d7a4429ed92ec79d
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [2]:
! git clone https://github.com/huggingface/lerobot.git

Cloning into 'lerobot'...
remote: Enumerating objects: 37682, done.
remote: Counting objects: 100% (829/829), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 37682 (delta 715), reused 535 (delta 524), pack-reused 36853 (from 4)
Receiving objects: 100% (37682/37682), 169.10 MiB | 19.89 MiB/s, done.
Resolving deltas: 100% (24349/24349), done.
Filtering content: 100% (45/45), 69.04 MiB | 54.97 MiB/s, done.


In [3]:
! cd lerobot && pip install .

Processing /content/lerobot
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 MB 15.4 MB/s et

In [ ]:
import re
import safetensors

_VARIANT_RE = re.compile(r"\.so\d+(?:-[\w]+)?_buffer_")

def canonicalise(k: str) -> str:
    """
    Remove dataset-variant markers like '.so100-blue_' or '.so100_' from a
    normalisation-buffer key.
    """
    return _VARIANT_RE.sub(".buffer_", k)

def standardise_state_dict(
    checkpoint: dict[str, torch.Tensor], ref_keys: set[str], *, verbose: bool = True
) -> tuple[dict[str, torch.Tensor], list[str]]:
    """
    • Re-keys `checkpoint ` so that every entry matches the *reference* key set.
    • If several variant keys collapse to the same canonical name we keep the
      first one and log the collision.
    • Returns the new dict + a list of entries that could not be matched.
    """
    out, collisions, unmatched = {}, {}, []

    for k, v in checkpoint.items():
        canon = canonicalise(k)
        if canon in ref_keys:
            if canon in out:  # duplicate after collapsing
                collisions.setdefault(canon, []).append(k)
            else:
                out[canon] = v
        else:
            unmatched.append(k)

    if verbose:
        for canon, variants in collisions.items():
            print(f"[standardise_state_dict] '{canon}'  ←  {variants}")
        if unmatched:
            print(f"[standardise_state_dict] kept {len(unmatched)} unmatched keys")

    out.update({k: checkpoint[k] for k in unmatched})
    return out, unmatched

def rename_checkpoint_keys(checkpoint: dict, rename_str: str):
    """
    Renames keys in a checkpoint dictionary based on the given rename string.

    Args:
        checkpoint (dict): The checkpoint dictionary.
        rename_str (str): A string specifying key mappings in the format "old1//new1,old2//new2".

    Returns:
        dict: The modified checkpoint with renamed keys.
    """

    rename_dict = dict(pair.split("//") for pair in rename_str.split(","))

    new_checkpoint = {}
    for k, v in checkpoint.items():
        for old_key, new_key in rename_dict.items():
            if old_key in k:
                k = k.replace(old_key, new_key)
        new_checkpoint[k] = v
    return new_checkpoint

def load_smolvla(
    model: torch.nn.Module,
    filename: str | os.PathLike,
    *,
    device: str = "cpu",
    checkpoint_keys_mapping: str = "",
) -> torch.nn.Module:
    state_dict = safetensors.torch.load_file(filename, device=device)

    # Optional user-supplied renames (e.g. "model._orig_mod.//model.")
    if checkpoint_keys_mapping and "//" in checkpoint_keys_mapping:
        state_dict = rename_checkpoint_keys(state_dict, checkpoint_keys_mapping)

    state_dict, _ = standardise_state_dict(state_dict, set(model.state_dict().keys()))

    # HACK(aliberts): to not overwrite normalization parameters as they should come from the dataset
    norm_keys = ("normalize_inputs", "normalize_targets", "unnormalize_outputs")
    state_dict = {k: v for k, v in state_dict.items() if not k.startswith(norm_keys)}

    missing, unexpected = model.load_state_dict(state_dict, strict=False)

    if not all(key.startswith(norm_keys) for key in missing) or unexpected:
        raise RuntimeError(
            "SmolVLA %d missing / %d unexpected keys",
            len(missing),
            len(unexpected),
        )

    return model

In [6]:
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig
import pprint
pprint.pprint(SmolVLAConfig.__dict__)


mappingproxy({'__abstractmethods__': frozenset(),
              '__annotations__': {'adapt_to_pi_aloha': <class 'bool'>,
                                  'add_image_special_tokens': <class 'bool'>,
                                  'attention_mode': <class 'str'>,
                                  'chunk_size': <class 'int'>,
                                  'empty_cameras': <class 'int'>,
                                  'expert_width_multiplier': <class 'float'>,
                                  'freeze_vision_encoder': <class 'bool'>,
                                  'load_vlm_weights': <class 'bool'>,
                                  'max_action_dim': <class 'int'>,
                                  'max_period': <class 'float'>,
                                  'max_state_dim': <class 'int'>,
                                  'min_period': <class 'float'>,
                                  'n_action_steps': <class 'int'>,
                                  'n_obs_steps': <cl

In [42]:
import torch
from torch import nn
from lerobot.policies.smolvla.smolvlm_with_expert import SmolVLMWithExpertModel
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig

class SmolVLMForONNXExport(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()

        self.config = SmolVLAConfig
        self.config.self_attn_every_n_layers = 1 # just test
        # Instantiate the real model
        # self.model = SmolVLMWithExpertModel(model_id=self.config.vlm_model_name,
                                            # attention_mode="cross_attn")
        self.model = SmolVLMWithExpertModel(model_id=self.config.vlm_model_name,
                                            freeze_vision_encoder=self.config.freeze_vision_encoder,
                                            train_expert_only=self.config.train_expert_only,
                                            load_vlm_weights=self.config.load_vlm_weights,
                                            attention_mode=self.config.attention_mode,
                                            num_expert_layers=self.config.num_expert_layers,
                                            num_vlm_layers=self.config.num_vlm_layers,
                                            self_attn_every_n_layers=self.config.self_attn_every_n_layers,
                                            expert_width_multiplier=self.config.expert_width_multiplier)

        # Bake in the configuration for the static graph
        self.attention_mode = self.config.vlm_model_name
        self.num_vlm_layers = self.model.num_vlm_layers
        self.num_expert_layers = self.model.num_expert_layers
        self.fill_kv_cache = False
        self.use_cache = True

    def forward(
          self,
          vlm_embeds: torch.Tensor,
          expert_embeds: torch.Tensor,
          attention_mask: torch.Tensor,
          position_ids: torch.LongTensor,
          # Flattened past_key_values: one tensor for keys, one for values per layer
          *past_key_values_flat: torch.Tensor):


          inputs_embeds = [vlm_embeds, expert_embeds]

          past_key_values = {}
          if past_key_values_flat:
              # Logic to reconstruct the dictionary from the flat tuple of tensors
              # This assumes a consistent ordering: (key_layer0, val_layer0, key_layer1, val_layer1, ...)
              for i in range(0, len(past_key_values_flat), 2):
                  layer_idx = i // 2
                  past_key_values[layer_idx] = {
                      "key_states": past_key_values_flat[i],
                      "value_states": past_key_values_flat[i+1],
                  }
          else:
              past_key_values = None


          fill_kv_cache = past_key_values is None


          final_embeds_list, new_past_key_values = self.model.forward(
              attention_mask=attention_mask,
              position_ids=position_ids,
              past_key_values=past_key_values,
              inputs_embeds=inputs_embeds,
              use_cache=self.use_cache,
              fill_kv_cache=self.fill_kv_cache,
          )

          # --- 3. Flatten Outputs ---
          # The output must be a flat tuple of tensors
          vlm_output_embeds = final_embeds_list[0]
          expert_output_embeds = final_embeds_list[1]

          present_key_values_flat = []
          if new_past_key_values:
            for i in range(self.num_vlm_layers):
                if i in new_past_key_values:
                    present_key_values_flat.append(new_past_key_values[i]["key_states"])
                    present_key_values_flat.append(new_past_key_values[i]["value_states"])

          return (vlm_output_embeds, expert_output_embeds, *present_key_values_flat)

In [ ]:
import torch


onnx_model = SmolVLMForONNXExport("HuggingFaceTB/SmolVLM2-500M-Video-Instruct")

In [9]:

# pprint.pprint(onnx_model.config.num_key_value_heads)
# exit(0)


onnx_model.eval()

onnx_model.fill_kv_cache = True

onnx_model.to("cuda:0")

output_names = ["vlm_output_embeds", "expert_output_embeds"]
num_layers = onnx_model.model.num_vlm_layers
for i in range(num_layers):
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size

batch_size = 1
vlm_seq_len = 256
expert_seq_len = 16
total_seq_len = vlm_seq_len + expert_seq_len

dummy_vlm_embeds = torch.randn(batch_size, vlm_seq_len, 960, device="cuda:0", dtype=torch.bfloat16)
# dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, 320, device="cuda:0", dtype=torch.bfloat16) # input_embeds
dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16) # input_embeds
dummy_attn_mask = torch.ones(batch_size, total_seq_len, total_seq_len, device="cuda:0", dtype=torch.bool)
dummy_pos_ids = torch.arange(total_seq_len, device="cuda:0").unsqueeze(0)


dummy_inputs = (dummy_vlm_embeds, dummy_expert_embeds, dummy_attn_mask, dummy_pos_ids)

# we ant past_key_values from it, the second output
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_prefill.onnx",
    input_names=["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"],
    output_names=output_names,
    opset_version=17
)


/usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/smolvlm_with_expert.py:237: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_len < position_ids.shape[1]:


In [10]:
onnx_model.config.attention_mode

'cross_attn'

In [11]:
%pip install -q ipdb
%env PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.1 MB/s eta 0:00:00
env: PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb


In [12]:
%pdb on

Automatic pdb calling has been turned ON


In [35]:
onnx_model.to("cuda:0")
onnx_model.fill_kv_cache = False
onnx_model.use_cache = False

In [36]:
onnx_model.use_cache

False

In [ ]:
import torch


onnx_model = SmolVLMForONNXExport("HuggingFaceTB/SmolVLM2-500M-Video-Instruct")
onnx_model.eval()

In [45]:

onnx_model.to("cuda:0")
onnx_model.fill_kv_cache = False
onnx_model.self_attn_every_n_layers = -1
onnx_model.use_cache = False
current_seq_len = 1
# past_seq_len = 256
past_seq_len = 11

batch_size = 1

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size
num_layers = onnx_model.model.num_vlm_layers
num_kv_heads = onnx_model.model.config.text_config.num_key_value_heads
head_dim = vlm_hidden_dim // onnx_model.model.config.text_config.num_attention_heads

dummy_vlm_embeds = torch.randn(batch_size, current_seq_len, 960, device="cuda:0", dtype=torch.bfloat16)
dummy_expert_embeds = torch.randn(batch_size, current_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16)
# dummy_expert_embeds = torch.randn(batch_size, current_seq_len, 720, device="cuda:0", dtype=torch.bfloat16)
# dummy_attn_mask = torch.ones(batch_size, total_seq_len, total_seq_len, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(1, past_seq_len + current_seq_len, past_seq_len + current_seq_len + 1, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len + current_seq_len, past_seq_len + current_seq_len + 1, device="cuda:0", dtype=torch.bool)
dummy_attn_mask = torch.ones(batch_size, past_seq_len + current_seq_len, past_seq_len + current_seq_len + 1, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len, past_seq_len + current_seq_len + 1, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len + current_seq_len, device="cuda:0", dtype=torch.bool)
dummy_pos_ids = torch.arange(past_seq_len + current_seq_len, device="cuda:0").unsqueeze(0)

print(past_seq_len + current_seq_len)
print(total_seq_len)

input_names = ["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"]

past_key_values_flat = []
for _ in range(num_layers):
    key_shape = (batch_size, num_kv_heads, past_seq_len, head_dim)
    val_shape = (batch_size, num_kv_heads, past_seq_len, head_dim)
    past_key_values_flat.append(torch.randn(key_shape, device="cuda:0", dtype=torch.bfloat16))
    past_key_values_flat.append(torch.randn(val_shape, device="cuda:0", dtype=torch.bfloat16))

dummy_inputs = (
    dummy_vlm_embeds,
    dummy_expert_embeds,
    dummy_attn_mask,
    dummy_pos_ids,
    *past_key_values_flat  # Unpack the list into the tuple
    # None
)


for i in range(num_layers):
    input_names.append(f"past_key_{i}")
    input_names.append(f"past_value_{i}")


# Outputs include the embeddings and the *updated* KV cache
output_names = ["vlm_output_embeds", "expert_output_embeds"]
for i in range(num_layers):
    # The output names must be consistent for the next iteration
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

dynamic_axes = {"vlm_embeds": {}, "expert_embeds": {}} # Add other static inputs if needed
for i in range(num_layers):
    # The '2' axis is the sequence length dimension in the KV cache shape
    # (batch_size, num_heads, sequence_length, head_dim)
    dynamic_axes[f"past_key_{i}"] = {2: "past_sequence_length"}
    dynamic_axes[f"past_value_{i}"] = {2: "past_sequence_length"}
    dynamic_axes[f"present_key_{i}"] = {2: "total_sequence_length"}
    dynamic_axes[f"present_value_{i}"] = {2: "total_sequence_length"}

# we need output_embeds - first output
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_decode.onnx",
    input_names=input_names,
    output_names=output_names,
    dynamic_axes=dynamic_axes,
    opset_version=17,
    do_constant_folding=True,
)


12
272


In [ ]:
import os, onnx

out_name = None
src_name = "smolvlm_expert_prefill.onnx"
out_dir = "splited"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )

In [ ]:
import os, onnx

out_name = None
src_name = "smolvlm_expert_decode.onnx"
out_dir = "splited_decoder"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )